In [1]:
from PIL import Image
import numpy as np
import os

In [2]:
def apply_mask(image_array):
    # Ensure the image array is square
    height = image_array.shape[0]
    width = image_array.shape[1]
    if height != width:
        raise ValueError("Image array must be square (n x n pixels).")

    # Create a circular mask
    radius = width // 2
    Y, X = np.ogrid[:height, :width]
    dist_from_center = np.sqrt((X+.5 - width / 2) ** 2 + (Y+.5 - height / 2) ** 2)
    mask = dist_from_center <= radius

    # Initialize an image array with an alpha channel
    masked_img = np.zeros((height, width), dtype=np.int8)  # 4 channels: R, G, B, and Alpha


    # Apply the mask to the alpha channel
    masked_img = image_array*mask
    return masked_img

In [3]:
def intensity_match(texture_array, output_array):
    
    new_output = np.zeros(output_array.shape,dtype=texture_array.dtype)
    texture_data = texture_array.flatten()
    output_data = output_array.flatten()
    
    sorted_output = np.argsort(output_data)
    sorted_texture = np.argsort(texture_data)
    
    new_output_data = np.zeros(output_data.shape,dtype=texture_array.dtype)
    new_output_data[sorted_output] = texture_data[sorted_texture]
    
    new_output = new_output_data.reshape(output_array.shape)

    return new_output

In [4]:
def post_process(reference_pgm, generated_jpg):
    img_0 = Image.open(reference_pgm)
    img_0_array = np.array(img_0)
    img_1 = Image.open(generated_jpg)
    img_1_array = np.array(img_1)
    img_1_array = img_1_array[:,:,0]
    img0_masked = apply_mask(img_0_array)
    img1_masked = apply_mask(img_1_array)
    img1_matched = intensity_match(img0_masked,img1_masked)
    ref_masked = Image.fromarray(img0_masked)
    ref_masked.save(os.path.splitext(reference_pgm)[0]+'_masked.pgm')
    gen_masked = Image.fromarray(img1_masked)
    gen_masked.save(os.path.splitext(generated_jpg)[0]+'_masked.pgm')
    gen_matched = Image.fromarray(img1_matched)
    gen_matched.save(os.path.splitext(generated_jpg)[0]+'_matched.pgm')

In [5]:
for i in [2,4,8]:
    input_file = "./image_resources/processed/rock-0_processed_1%d.pgm"%i
    for h in range(3):
        output_file = "./image_resources/outputs/rock-0_%d_%d_C5_final.jpg"%(h,i)
        post_process(input_file, output_file)